## PaySim Dataset

##### <b>step</b> - integer - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

##### <b>type</b> - string/categorical - type of transaction: CASH_IN, CASH_OUT, DEBIT, PAYMENT and TRANSFER.

##### <b>amount</b> - float - amount of the transaction in local currency.

##### <b>nameOrig</b> - string - customer who initiated the transaction

##### <b>oldbalanceOrg</b> - float initial balance before the transaction

##### <b>newbalanceOrig</b> - float - new balance after the transaction

##### <b>nameDest</b> - string - customer who is the recipient of the transaction

##### <b>oldbalanceDest</b> - float - initial balance of recipient before the transaction.

##### <b>newbalanceDest</b> - float - new balance of recipient after the transaction.

##### <b>fraud</b> - boolean/binary - determines if transaction is fraudulent

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
3,application_1606928535611_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from graphframes import *
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

An error was encountered:
No module named 'graphframes'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'graphframes'



In [3]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

def extract_node_type(x):
    if (x.startswith("C")):
        node_type = 0
    elif (x.startswith("B")):
        node_type = 1
    elif (x.startswith("M")):
        node_type = 2
    else:
        node_type = 99
    return node_type

def extract_fraudster(x):
    if (x.startswith("CF")):
        fraudster = 1
    else:
        fraudster = 0
    return fraudster

def action_2_code(x):
    if (x == "CASH_IN"):
        node_type = 0
    elif (x == "CASH_OUT"):
        node_type = 1
    elif (x == "DEBIT"):
        node_type = 2
    elif (x == "PAYMENT"):
        node_type = 3
    elif (x == "TRANSFER"):
        node_type = 4
    elif (x == "DEPOSIT"):
        node_type = 4        
    else:
        node_type = 99
    return node_type

def gender_2_code(x):
    if x == 'W':
        gender =  0
    elif x == 'M':
        gender = 1
    else: 
        gender = 99
    return gender

def timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    return dt_obj.timestamp()

hashnode_udf = func.udf(hashnode)
extract_fraudster_udf = func.udf(extract_fraudster)
node_type_udf = func.udf(extract_node_type)
action_2_code_udf = func.udf(action_2_code)
gender_2_code_udf = func.udf(gender_2_code)
timestamp_2_time_udf = func.udf(timestamp_2_time)

An error was encountered:
name 'func' is not defined
Traceback (most recent call last):
NameError: name 'func' is not defined



In [4]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

An error was encountered:
name 'hsfs' is not defined
Traceback (most recent call last):
NameError: name 'hsfs' is not defined



In [5]:
transactions_fg = fs.get_feature_group("aml_10k_transactions_fg", 1)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined



In [6]:
transactions_fg.show(5)

An error was encountered:
name 'transactions_fg' is not defined
Traceback (most recent call last):
NameError: name 'transactions_fg' is not defined



In [7]:
transactions_df = transactions_fg.read()
transactions_df = transactions_df.withColumn('is_sar',func.when(func.col('is_sar') == 'true', 1).otherwise(0))\
                                 .withColumn('tx_type', action_2_code_udf(func.col('tx_type')))\
                                 .withColumn('tran_timestamp', timestamp_2_time_udf(func.col('tran_timestamp')).cast(FloatType()))\
                                 .withColumnRenamed("orig_acct","source")\
                                 .withColumnRenamed("bene_acct","target")\
                                 .select("source","target","tran_timestamp","is_sar","alert_id","tran_id","tx_type","base_amt")
transactions_df.show()
transactions_df.printSchema()

An error was encountered:
name 'transactions_fg' is not defined
Traceback (most recent call last):
NameError: name 'transactions_fg' is not defined



In [8]:
accounts_fg = fs.get_feature_group("aml_10k_accounts_fg", 1)
accounts_fg.show(5)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined



In [9]:
accounts_df = accounts_fg.read()
accounts_df = accounts_df.withColumn('prior_sar',func.when(func.col('prior_sar_count') == 'true', 1).otherwise(0))\
                         .withColumn('gender', gender_2_code_udf(func.col('gender')))\
                         .drop("prior_sar_count","acct_rptng_crncy","type","acct_stat","open_dt","bank_id","country","close_dt","dsply_nm","branch_id")
        

accounts_df.show(5)
accounts_df.printSchema()

An error was encountered:
name 'accounts_fg' is not defined
Traceback (most recent call last):
NameError: name 'accounts_fg' is not defined



In [10]:
edge_fg_meta = fs.create_feature_group(name="edge_features",
                                       version=1,
                                       primary_key=["tran_id"],
                                       description="edge features",
                                       time_travel_format=None,                                        
                                       statistics_config=False)
edge_fg_meta.save(transactions_df)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined



In [11]:
node_fg_meta = fs.create_feature_group(name="node_features",
                                              version=1,
                                              primary_key=["acct_id"],
                                              description="node features",
                                              time_travel_format=None,                                        
                                              statistics_config=False)
node_fg_meta.save(accounts_df)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined



In [12]:
edge_td_meta = fs.create_training_dataset(name="edge_td",
                                       version=1,
                                       data_format="csv",   
                                       description="edge td")
edge_td_meta.save(transactions_df)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined



In [13]:
node_td_meta = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="edge td")
node_td_meta.save(accounts_df)

An error was encountered:
name 'fs' is not defined
Traceback (most recent call last):
NameError: name 'fs' is not defined

